# 서울특별시 자치구별 버스 승하차인원 파악

## 목차 

### 1. module import
### 2. 정류소 좌표값을 이용해 해당 정류소가 속한 자치구 파악
### 3. 기간에 따른 정류소별 승하차 인원 확인
<hr/>

### 1. module import
- import **`pandas`** & import **`numpy`**
    - csv file 및 dataframe을 가공하기 위함
- import **`requests`** & import **`json`**
    - requests 모듈과 API를 활용하여 웹사이트에 접근하여,
    - 요청과 응답을 통해 json형태로 데이터를 받아온 후,
    - Python을 통해 필요한 데이터를 수집하고 가공하기위함

In [1]:
import pandas as pd
import numpy as np
import requests
import json

### 2. 정류소별 좌표값을 이용해 정류소가 속한 자치구 파악

#### 2.1 pandas를 활용해 csv파일 불러오기 및 dataframe 구조 확인

In [2]:
# 정류소별 좌표데이터 파일 불러오기
coor = pd.read_csv("./data/bus/서울시버스정류소좌표데이터(2022.08.24).csv", encoding="euc-kr")

In [3]:
# 데이터 확인
coor.head(5)

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y
0,100000001,1001,종로2가사거리,126.987786,37.569764
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709
3,100000004,1004,종로2가.삼일교,126.987507,37.568582
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230


In [4]:
# 데이터 구조 확인
coor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11167 entries, 0 to 11166
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   NODE_ID  11167 non-null  int64  
 1   ARS-ID   11167 non-null  int64  
 2   정류소명     11167 non-null  object 
 3   좌표X      11167 non-null  float64
 4   좌표Y      11167 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 436.3+ KB


In [5]:
# copy 만들어서 data 가공
my_coor = coor.copy()

In [6]:
# 좌표값 확인
my_coor["좌표X"].iloc[0]

126.9877861534

#### 2.2 네이버 클라우드 플랫폼의 Reverse-Geocoding application을 이용하여 좌표별 자치구 파악
- <a style="text-decoration: none; color:black;">[https://console.ncloud.com/naver-service/application]<a/>

In [7]:
# 발급받은 두개의 Application Key 저장
client_id = "8la6jz3flf"
client_secret = "jnuz1aeEGOEDBExyNsfmrfCZco9LDhuSLj36N9mb"
# 정류소별 자치구를 담을 공간
gu = []
for i in range(len(my_coor)):
    # 좌표값 변수에 담아주기 / 좌표X: 경도(longitude), 좌표Y: 위도(latitude)
    coords = str(my_coor["좌표X"].iloc[i])+","+str(my_coor["좌표Y"].iloc[i])
    # 요청을 보낼 url
    url=f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={coords}&output=json"
    # headers에 보낼 API-KEY
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }
    # url과 headers를 통해 서버에 request하고 response를 res에 저장
    res = requests.get(url, headers=headers)
    # json type의 데이터를 dict type으로 바꾸기
    my_dict = res.json()
    # Indexing을 통해 전체 주소 중 자치구만 gu에 추가
    gu.append(my_dict["results"][0]["region"]["area2"]["name"])

In [8]:
# "구" 라는 컬럼을 만들어 각 정류소가 속하는 자치구 추가
my_coor["구"] = gu

In [9]:
# "구" 컬럼이 잘 생성되었는지 확인
my_coor.head(5)

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y,구
0,100000001,1001,종로2가사거리,126.987786,37.569764,종로구
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179,종로구
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709,종로구
3,100000004,1004,종로2가.삼일교,126.987507,37.568582,종로구
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230,종로구


In [10]:
# 반복적인 API요청을 피하기 위해서 plus_gu.csv 파일로 저장
my_coor.to_csv("plus_gu.csv")

In [11]:
# 불러와서 체크
gu = pd.read_csv("./data/plus_gu.csv", index_col=0)

In [12]:
# 데이터 확인
# 필요한 컬럼은 ["ARS-ID", "구"]
gu.head()

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y,구
0,100000001,1001,종로2가사거리,126.987786,37.569764,종로구
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179,종로구
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709,종로구
3,100000004,1004,종로2가.삼일교,126.987507,37.568582,종로구
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230,종로구


In [13]:
# 불필요 컬럼 제거 및 재저장
gu.drop(["NODE_ID", "정류소명", "좌표X", "좌표Y"], axis = 1, inplace=True)
gu.to_csv("bus_gu.csv")

In [14]:
# 불러오기
gu = pd.read_csv("./bus_gu.csv", index_col=0)
gu.head()

,ARS-ID,구
0,1001,종로구
1,1002,종로구
2,1003,종로구
3,1004,종로구
4,1005,종로구


### 3. 기간에 따른 정류소별 승하차 인원 확인

#### 3.1 반복문을 통해 데이터를 변수에 저장
- 설정기간: 19년 1월 ~ 22년 9월

In [20]:
# ./data/bus 경로안에 BUS_STATION_BOARDING_MONTH_201901.csv 파일 형태로 저장되어있음
# exec 함수를 사용하여 동적할당을 통해 df19_01 형태의 변수로 저장(19년 1월 예시)

# data 상대경로 
url = "./data/bus/BUS_STATION_BOARDING_MONTH_20"
# encoding type
enc_type = "euc-kr"

# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지 밖에 없음
        for j in range(1, 10):
            url_temp = ""
            url_temp = f"{url}{i}0{j}.csv"
            exec(f"df{i}_0{j} = pd.read_csv(url_temp, encoding=enc_type)")
            print(url_temp) # 실행여부 확인
    else:
        for j in range(1, 13):
            url_temp = ""
            if j < 10:
                url_temp = f"{url}{i}0{j}.csv"
                exec(f"df{i}_0{j} = pd.read_csv(url_temp, encoding=enc_type)")
                print(url_temp)
            else:
                url_temp = f"{url}{i}{j}.csv"
                exec(f"df{i}_{j} = pd.read_csv(url_temp, encoding=enc_type)")
                print(url_temp)

./data/bus/BUS_STATION_BOARDING_MONTH_201901.csv
./data/bus/BUS_STATION_BOARDING_MONTH_201902.csv
./data/bus/BUS_STATION_BOARDING_MONTH_201903.csv
./data/bus/BUS_STATION_BOARDING_MONTH_201904.csv
./data/bus/BUS_STATION_BOARDING_MONTH_201905.csv
./data/bus/BUS_STATION_BOARDING_MONTH_201906.csv
./data/bus/BUS_STATION_BOARDING_MONTH_201907.csv
./data/bus/BUS_STATION_BOARDING_MONTH_201908.csv
./data/bus/BUS_STATION_BOARDING_MONTH_201909.csv
./data/bus/BUS_STATION_BOARDING_MONTH_201910.csv
./data/bus/BUS_STATION_BOARDING_MONTH_201911.csv
./data/bus/BUS_STATION_BOARDING_MONTH_201912.csv
./data/bus/BUS_STATION_BOARDING_MONTH_202001.csv
./data/bus/BUS_STATION_BOARDING_MONTH_202002.csv
./data/bus/BUS_STATION_BOARDING_MONTH_202003.csv
./data/bus/BUS_STATION_BOARDING_MONTH_202004.csv
./data/bus/BUS_STATION_BOARDING_MONTH_202005.csv
./data/bus/BUS_STATION_BOARDING_MONTH_202006.csv
./data/bus/BUS_STATION_BOARDING_MONTH_202007.csv
./data/bus/BUS_STATION_BOARDING_MONTH_202008.csv
./data/bus/BUS_STATI

In [21]:
# 데이터 확인
# 필요한 컬럼: ["버스정류장ARS번호", "승차총승객수", "하차총승객수"]
df19_01.head()

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20190101,100,100번(하계동~용산구청),11428,한성여객종점,10,0,20190104
1,20190101,100,100번(하계동~용산구청),11387,노원평생학습관,5,0,20190104
2,20190101,100,100번(하계동~용산구청),11373,중계역,27,1,20190104
3,20190101,100,100번(하계동~용산구청),11381,중계목화아파트4단지,110,12,20190104
4,20190101,100,100번(하계동~용산구청),11311,인덕대학,87,76,20190104


#### 3.2 과정 진행 중 문제점 및 해결방안
- df시리즈와 gu데이터의 공통 컬럼인 ARS(df: "버스정류장ARS번호", gu: "ARS-ID")를 통해 자치구별 승하차승객수를 구하고자함


- "버스정류장ARS번호"와 "ARS-ID"는 서로 데이터 타입이 다름 
    - gu데이터 기준으로 통일(ex. 07550(str) -> 7550(int))


- "버스정류장ARS번호" 컬럼에는 값이 '~'로 저장된 케이스가 있음
    - "~"에 해당하는 승하차승객수와 나머지를 비교해봤을 때 비교적 작은 수이기 때문에 해당 행들은 삭제

In [22]:
# 버스정류장ARS번호가 "~"에 해당하는 승객수와 나머지 비교
test01 = df22_09[df22_09["버스정류장ARS번호"] == "~"]
test02 = df22_09[df22_09["버스정류장ARS번호"] != "~"]

s1 = test01["승차총승객수"].sum() + test01["하차총승객수"].sum()
s2 = test02["승차총승객수"].sum() + test02["하차총승객수"].sum()
print(s1, s2)
# 274603 264466617 -> 유의미한 차이가 있기때문에 제거하기로 결정

274603 264466617


In [23]:
# exec를 통해 동적할당을 하기위한 변수 생성 
drop_columns = ["노선번호", "노선명", "등록일자"]
drop_columns2 = ["사용일자", "ARS-ID", "버스정류장ARS번호"]
ars_column = "버스정류장ARS번호"
new_column = "ARS-ID"
merge_type = "left"

# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지
        for j in range(1, 10):
            # 불필요한 컬럼 삭제
            # df.drop(["노선번호", "노선명", "등록일자"], axis=1, inplace=True)
            exec(f"df{i}_0{j}.drop(drop_columns, axis=1, inplace=True)")
            
            # ARS번호가 "~"이 아닌 행만 추출
            exec(f"df{i}_0{j} = df{i}_0{j}[df{i}_0{j}[ars_column] != '~']")
            
            # index 초기화
            # df = df.reset_index(drop=True)
            exec(f"df{i}_0{j} = df{i}_0{j}.reset_index(drop=True)")
            
            # ARS번호를 int형태로 바꾸기
            # df["ARS-ID"] = pd.to_numeric(df["버스정류장ARS번호"]) 
            exec(f"df{i}_0{j}[new_column] = pd.to_numeric(df{i}_0{j}[ars_column])")
            
            # ARS-ID를 기준으로 df와 gu데이터 병합
            # test = pd.merge(df, gu, how="left")
            exec(f"df{i}_0{j} = pd.merge(df{i}_0{j}, gu, how=merge_type)")
            
            # 필요 컬럼제외 전부 삭제
            exec(f"df{i}_0{j}.drop(drop_columns2, axis=1, inplace=True)")
            # 실행여부 확인
            print(f"df{i}_0{j}")
    else:
        for j in range(1, 13):
            if j < 10:
                exec(f"df{i}_0{j}.drop(drop_columns, axis=1, inplace=True)")
                exec(f"df{i}_0{j} = df{i}_0{j}[df{i}_0{j}[ars_column] != '~']")
                exec(f"df{i}_0{j} = df{i}_0{j}.reset_index(drop=True)")
                exec(f"df{i}_0{j}[new_column] = pd.to_numeric(df{i}_0{j}[ars_column])")
                exec(f"df{i}_0{j} = pd.merge(df{i}_0{j}, gu, how=merge_type)")
                exec(f"df{i}_0{j}.drop(drop_columns2, axis=1, inplace=True)")
                print(f"df{i}_0{j}")
            else:
                exec(f"df{i}_{j}.drop(drop_columns, axis=1, inplace=True)")
                exec(f"df{i}_{j} = df{i}_{j}[df{i}_{j}[ars_column] != '~']")
                exec(f"df{i}_{j} = df{i}_{j}.reset_index(drop=True)")
                exec(f"df{i}_{j}[new_column] = pd.to_numeric(df{i}_{j}[ars_column])")
                exec(f"df{i}_{j} = pd.merge(df{i}_{j}, gu, how=merge_type)")
                exec(f"df{i}_{j}.drop(drop_columns2, axis=1, inplace=True)")
                print(f"df{i}_{j}")
                
# 2205부터 표준버스정류장ID 컬럼 존재 -> 삭제
drop_column = ["표준버스정류장ID"]
for i in range(5, 10):
    exec(f"df22_0{i}.drop(drop_column, axis=1, inplace=True)")
    print(f"df22_0{i}")

df19_01
df19_02
df19_03
df19_04
df19_05
df19_06
df19_07
df19_08
df19_09
df19_10
df19_11
df19_12
df20_01
df20_02
df20_03
df20_04
df20_05
df20_06
df20_07
df20_08
df20_09
df20_10
df20_11
df20_12
df21_01
df21_02
df21_03
df21_04
df21_05
df21_06
df21_07
df21_08
df21_09
df21_10
df21_11
df21_12
df22_01
df22_02
df22_03
df22_04
df22_05
df22_06
df22_07
df22_08
df22_09
df22_05
df22_06
df22_07
df22_08
df22_09


In [24]:
# 데이터에서 "구" 컬럼이 들어간 것을 확인
df22_09.head()

,역명,승차총승객수,하차총승객수,구
0,창경궁.서울대학교병원(00031),101,176,종로구
1,광화문광화문빌딩(00045),220,299,종로구
2,사직단(00042),129,310,종로구
3,서울경찰청.경복궁역(00043),168,420,종로구
4,세종문화회관(00044),98,250,종로구


#### 3.3 자치구별로 그룹화하여 하차총승객수로 내림차순 정리 및 csv파일로 저장

In [25]:
# exec를 통해 동적할당을 하기위한 변수 생성 
guu = "구"
hacha = "하차총승객수"
bus = "bus"
col_name = "기간"

# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지
        for j in range(1, 10):
            # "구"를 기준으로 groupby
            exec(f"df{i}_0{j} = df{i}_0{j}.groupby(guu).sum()")  
            
            # "하차총승객수"를 기준으로 내림차순 정리
            exec(f"df{i}_0{j} = df{i}_0{j}.sort_values(hacha, ascending=False)")
            
            # 데이터에 몇년도 몇월 자료인지 "기간" 컬럼 추가
            date = f"20{i}0{j}"
            exec(f"df{i}_0{j}[col_name] = date")
            
            # 데이터를 bus2209.csv 파일형태로 저장(ex. 22년 9월)
            csv_name = f"bus20{i}0{j}.csv"
            exec(f"df{i}_0{j}.to_csv(csv_name)")
            
            # 실행여부 확인
            print(f"df{i}_0{j}")
    else:
        for j in range(1, 13):
            if j < 10:
                exec(f"df{i}_0{j} = df{i}_0{j}.groupby(guu).sum()")  
                exec(f"df{i}_0{j} = df{i}_0{j}.sort_values(hacha, ascending=False)")
                date = f"20{i}0{j}"
                exec(f"df{i}_0{j}[col_name] = date")
                csv_name = f"bus20{i}0{j}.csv"
                exec(f"df{i}_0{j}.to_csv(csv_name)")
                print(f"df{i}_0{j}")
            else:
                exec(f"df{i}_{j} = df{i}_{j}.groupby(guu).sum()")  
                exec(f"df{i}_{j} = df{i}_{j}.sort_values(hacha, ascending=False)")
                date = f"20{i}{j}"
                exec(f"df{i}_{j}[col_name] = date")
                csv_name = f"bus20{i}{j}.csv"
                exec(f"df{i}_{j}.to_csv(csv_name)")
                print(f"df{i}_{j}")

df19_01
df19_02
df19_03
df19_04
df19_05
df19_06
df19_07
df19_08
df19_09
df19_10
df19_11
df19_12
df20_01
df20_02
df20_03
df20_04
df20_05
df20_06
df20_07
df20_08
df20_09
df20_10
df20_11
df20_12
df21_01
df21_02
df21_03
df21_04
df21_05
df21_06
df21_07
df21_08
df21_09
df21_10
df21_11
df21_12
df22_01
df22_02
df22_03
df22_04
df22_05
df22_06
df22_07
df22_08
df22_09


#### 3.4 저장한 데이터 읽어오기 및 컬럼 순서 변경

In [26]:
# 저장한 데이터 읽어오기

# 상대경로
url = "./data/busbus/bus"
# enc_type = "euc-kr"
# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지 밖에 없음
        for j in range(1, 10):
            url_temp = f"{url}{i}0{j}.csv"
            exec(f"bus{i}0{j} = pd.read_csv(url_temp, index_col=0)")
            print(url_temp)
    else:
        for j in range(1, 13):
            if j < 10:
                url_temp = f"{url}{i}0{j}.csv"
                exec(f"bus{i}0{j} = pd.read_csv(url_temp, index_col=0)")
                print(url_temp)
            else:
                url_temp = f"{url}{i}{j}.csv"
                exec(f"bus{i}{j} = pd.read_csv(url_temp, index_col=0)")
                print(url_temp)

./data/busbus/bus1901.csv
./data/busbus/bus1902.csv
./data/busbus/bus1903.csv
./data/busbus/bus1904.csv
./data/busbus/bus1905.csv
./data/busbus/bus1906.csv
./data/busbus/bus1907.csv
./data/busbus/bus1908.csv
./data/busbus/bus1909.csv
./data/busbus/bus1910.csv
./data/busbus/bus1911.csv
./data/busbus/bus1912.csv
./data/busbus/bus2001.csv
./data/busbus/bus2002.csv
./data/busbus/bus2003.csv
./data/busbus/bus2004.csv
./data/busbus/bus2005.csv
./data/busbus/bus2006.csv
./data/busbus/bus2007.csv
./data/busbus/bus2008.csv
./data/busbus/bus2009.csv
./data/busbus/bus2010.csv
./data/busbus/bus2011.csv
./data/busbus/bus2012.csv
./data/busbus/bus2101.csv
./data/busbus/bus2102.csv
./data/busbus/bus2103.csv
./data/busbus/bus2104.csv
./data/busbus/bus2105.csv
./data/busbus/bus2106.csv
./data/busbus/bus2107.csv
./data/busbus/bus2108.csv
./data/busbus/bus2109.csv
./data/busbus/bus2110.csv
./data/busbus/bus2111.csv
./data/busbus/bus2112.csv
./data/busbus/bus2201.csv
./data/busbus/bus2202.csv
./data/busbu

In [27]:
# 컬럼순서 변경
new_columns = ["기간", "구", "승차총승객수", "하차총승객수"]

# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지
        for j in range(1, 10):
            exec(f"bus{i}0{j} = bus{i}0{j}[new_columns]")  
            print(f"bus{i}0{j}")
    else:
        for j in range(1, 13):
            if j < 10:
                exec(f"bus{i}0{j} = bus{i}0{j}[new_columns]")  
                print(f"bus{i}0{j}")
            else:
                exec(f"bus{i}{j} = bus{i}{j}[new_columns]")  
                print(f"bus{i}{j}")

bus1901
bus1902
bus1903
bus1904
bus1905
bus1906
bus1907
bus1908
bus1909
bus1910
bus1911
bus1912
bus2001
bus2002
bus2003
bus2004
bus2005
bus2006
bus2007
bus2008
bus2009
bus2010
bus2011
bus2012
bus2101
bus2102
bus2103
bus2104
bus2105
bus2106
bus2107
bus2108
bus2109
bus2110
bus2111
bus2112
bus2201
bus2202
bus2203
bus2204
bus2205
bus2206
bus2207
bus2208
bus2209


In [28]:
# 데이터 확인
bus2209

,기간,구,승차총승객수,하차총승객수
0,2209,강남구,8790206,8174799
1,2209,관악구,7653326,7786523
2,2209,성북구,6996266,6975473
3,2209,서초구,6477203,6403860
4,2209,서대문구,6189441,6130893
5,2209,영등포구,6176261,6114386
6,2209,종로구,5957177,5635348
7,2209,송파구,5678214,5579323
8,2209,동대문구,5513893,5379818
9,2209,마포구,5487622,5340607


In [29]:
# 서울특별시의 25개 자치구만 추출(불필요한 행 제거)
# 전체 데이터 확인 결과 0~24번째 인덱스를 제외한 행은 불필요
# 추출 후 csv 파일로 저장

for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지
        for j in range(1, 10):
            exec(f"bus{i}0{j} = bus{i}0{j}.iloc[:25]")  
            csv_name = f"bus{i}0{j}.csv"
            exec(f"bus{i}0{j}.to_csv(csv_name)")
            print(f"bus{i}0{j}")
    else:
        for j in range(1, 13):
            if j < 10:
                exec(f"bus{i}0{j} = bus{i}0{j}.iloc[:25]")
                csv_name = f"bus{i}0{j}.csv"
                exec(f"bus{i}0{j}.to_csv(csv_name)")
                print(f"bus{i}0{j}")
            else:
                exec(f"bus{i}{j} = bus{i}{j}.iloc[:25]")  
                csv_name = f"bus{i}{j}.csv"
                exec(f"bus{i}{j}.to_csv(csv_name)")
                print(f"bus{i}{j}")

bus1901
bus1902
bus1903
bus1904
bus1905
bus1906
bus1907
bus1908
bus1909
bus1910
bus1911
bus1912
bus2001
bus2002
bus2003
bus2004
bus2005
bus2006
bus2007
bus2008
bus2009
bus2010
bus2011
bus2012
bus2101
bus2102
bus2103
bus2104
bus2105
bus2106
bus2107
bus2108
bus2109
bus2110
bus2111
bus2112
bus2201
bus2202
bus2203
bus2204
bus2205
bus2206
bus2207
bus2208
bus2209


In [30]:
bus2209.head()

,기간,구,승차총승객수,하차총승객수
0,2209,강남구,8790206,8174799
1,2209,관악구,7653326,7786523
2,2209,성북구,6996266,6975473
3,2209,서초구,6477203,6403860
4,2209,서대문구,6189441,6130893


#### 기타작업
- 월별로 일수가 제각각이기 때문에 월 평균데이터도 구하기

In [31]:
# 월 평균 구하기(ex 1월: 31로 나눠주기)
# 20년 2월은 28일까지 있음
# 1,3,5,7,8,10,12월: 31일
# 4,6,9,11월: 30일
# test[["승차총승객수"]] = test[["승차총승객수"]] // 30
ls_31 = [1, 3, 5, 7, 8, 10, 12]
ls_30 = [4, 6, 9, 11]

# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지
        for j in range(1, 10):
            exec(f"data = bus{i}0{j}.copy()")
            # 31일까지 있는 달
            if j in ls_31:
                data[["승차총승객수"]] = data[["승차총승객수"]] // 31
                data[["하차총승객수"]] = data[["하차총승객수"]] // 31
                # 기간명 변경
                data["기간"] = f"20{i}0{j}"
            # 30일까지 있는 달
            elif j in ls_30:
                data[["승차총승객수"]] = data[["승차총승객수"]] // 30
                data[["하차총승객수"]] = data[["하차총승객수"]] // 30
                data["기간"] = f"20{i}0{j}"
            # 2월
            elif j == 2:
                data[["승차총승객수"]] = data[["승차총승객수"]] // 28
                data[["하차총승객수"]] = data[["하차총승객수"]] // 28
                data["기간"] = f"20{i}0{j}"
            bus = data.copy()
            csv_name = f"bus20{i}0{j}_avg.csv"
            bus.to_csv(csv_name)
    elif i == 20:
        for j in range(1, 13):
            if j < 10:
                exec(f"data = bus{i}0{j}.copy()")
                if j in ls_31:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 31
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 31
                    data["기간"] = f"20{i}0{j}"
                elif j in ls_30:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 30
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 30
                    data["기간"] = f"20{i}0{j}"
                elif j == 2:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 29
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 29
                    data["기간"] = f"20{i}0{j}"
                bus = data.copy()
                csv_name = f"bus20{i}0{j}_avg.csv"
                bus.to_csv(csv_name)
            else:
                exec(f"data = bus{i}{j}.copy()")
                if j in ls_31:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 31
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 31
                    data["기간"] = f"20{i}{j}"
                elif j in ls_30:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 30
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 30
                    data["기간"] = f"20{i}{j}"
                bus = data.copy()
                csv_name = f"bus20{i}{j}_avg.csv"
                bus.to_csv(csv_name)
    else:
        for j in range(1, 13):
            if j < 10:
                exec(f"data = bus{i}0{j}.copy()")
                if j in ls_31:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 31
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 31
                    data["기간"] = f"20{i}0{j}"
                elif j in ls_30:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 30
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 30
                    data["기간"] = f"20{i}0{j}"
                elif j == 2:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 28
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 28
                    data["기간"] = f"20{i}0{j}"
                bus = data.copy()
                csv_name = f"bus20{i}0{j}_avg.csv"
                bus.to_csv(csv_name)
            else:
                exec(f"data = bus{i}{j}.copy()")
                if j in ls_31:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 31
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 31
                    data["기간"] = f"20{i}{j}"
                elif j in ls_30:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 30
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 30
                    data["기간"] = f"20{i}{j}"
                bus = data.copy()
                csv_name = f"bus20{i}{j}_avg.csv"
                bus.to_csv(csv_name)

In [32]:
# 저장한 데이터 읽어오기

# 상대경로
url = "./data/bus_avg/bus"
# enc_type = "euc-kr"
# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지 밖에 없음
        for j in range(1, 10):
            url_temp = f"{url}20{i}0{j}_avg.csv"
            exec(f"bus20{i}0{j}_avg = pd.read_csv(url_temp, index_col=0)")
            print(url_temp)
    else:
        for j in range(1, 13):
            if j < 10:
                url_temp = f"{url}20{i}0{j}_avg.csv"
                exec(f"bus20{i}0{j}_avg = pd.read_csv(url_temp, index_col=0)")
                print(url_temp)
            else:
                url_temp = f"{url}20{i}{j}_avg.csv"
                exec(f"bus20{i}{j}_avg = pd.read_csv(url_temp, index_col=0)")
                print(url_temp)

./data/bus_avg/bus201901_avg.csv
./data/bus_avg/bus201902_avg.csv
./data/bus_avg/bus201903_avg.csv
./data/bus_avg/bus201904_avg.csv
./data/bus_avg/bus201905_avg.csv
./data/bus_avg/bus201906_avg.csv
./data/bus_avg/bus201907_avg.csv
./data/bus_avg/bus201908_avg.csv
./data/bus_avg/bus201909_avg.csv
./data/bus_avg/bus201910_avg.csv
./data/bus_avg/bus201911_avg.csv
./data/bus_avg/bus201912_avg.csv
./data/bus_avg/bus202001_avg.csv
./data/bus_avg/bus202002_avg.csv
./data/bus_avg/bus202003_avg.csv
./data/bus_avg/bus202004_avg.csv
./data/bus_avg/bus202005_avg.csv
./data/bus_avg/bus202006_avg.csv
./data/bus_avg/bus202007_avg.csv
./data/bus_avg/bus202008_avg.csv
./data/bus_avg/bus202009_avg.csv
./data/bus_avg/bus202010_avg.csv
./data/bus_avg/bus202011_avg.csv
./data/bus_avg/bus202012_avg.csv
./data/bus_avg/bus202101_avg.csv
./data/bus_avg/bus202102_avg.csv
./data/bus_avg/bus202103_avg.csv
./data/bus_avg/bus202104_avg.csv
./data/bus_avg/bus202105_avg.csv
./data/bus_avg/bus202106_avg.csv
./data/bus

#### 3.5 모든 bus data 병합

In [33]:
bus_data = [bus1901,bus1902,bus1903,bus1904,bus1905,bus1906,bus1907,bus1908,bus1909,bus1910,bus1911,bus1912,bus2001,bus2002,bus2003,bus2004,
           bus2005,bus2006,bus2007,bus2008,bus2009,bus2010,bus2011,bus2012,bus2101,bus2102,bus2103,bus2104,bus2105,bus2106,bus2107,bus2108,
           bus2109,bus2110,bus2111,bus2112,bus2201,bus2202,bus2203,bus2204,bus2205,bus2206,bus2207,bus2208,bus2209]
result = pd.concat(bus_data, axis=0).reset_index(drop=True)

In [34]:
result

,기간,구,승차총승객수,하차총승객수
0,1901,강남구,10359360,9624393
1,1901,관악구,9043790,9125741
2,1901,서초구,8085917,7830620
3,1901,성북구,7314353,7293126
4,1901,영등포구,6928340,6979443
...,...,...,...,...
1120,2209,금천구,3714193,3575138
1121,2209,도봉구,3414409,3261866
1122,2209,성동구,2925575,2865302
1123,2209,광진구,2855455,2860857


In [35]:
# csv 파일로 저장
result.to_csv("bus_result.csv")

In [36]:
bus_data = [bus201901_avg,bus201902_avg,bus201903_avg,bus201904_avg,bus201905_avg,bus201906_avg,bus201907_avg,bus201908_avg,bus201909_avg,bus201910_avg,bus201911_avg,bus201912_avg,
            bus202001_avg,bus202002_avg,bus202003_avg,bus202004_avg,bus202005_avg,bus202006_avg,bus202007_avg,bus202008_avg,bus202009_avg,bus202010_avg,bus202011_avg,bus202012_avg,
            bus202101_avg,bus202102_avg,bus202103_avg,bus202104_avg,bus202105_avg,bus202106_avg,bus202107_avg,bus202108_avg,bus202109_avg,bus202110_avg,bus202111_avg,bus202112_avg,
            bus202201_avg,bus202202_avg,bus202203_avg,bus202204_avg,bus202205_avg,bus202206_avg,bus202207_avg,bus202208_avg,bus202209_avg]
result = pd.concat(bus_data, axis=0)

In [39]:
result_avg = result.reset_index(drop=True)

In [38]:
result_avg.to_csv("bus_result_avg.csv")

In [40]:
result_avg

,기간,구,승차총승객수,하차총승객수
0,201901,강남구,334172,310464
1,201901,관악구,291735,294378
2,201901,서초구,260836,252600
3,201901,성북구,235946,235262
4,201901,영등포구,223494,225143
...,...,...,...,...
1120,202209,금천구,123806,119171
1121,202209,도봉구,113813,108728
1122,202209,성동구,97519,95510
1123,202209,광진구,95181,95361
